In [12]:
#currency_Recon_VCOMP Pipeline
#Libray Imports
import os
import numpy as np
import cv2
from glob import glob
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from pathlib import Path

#data_folder = Path("source_data/text_files/")

#file_to_open = data_folder / "raw_data.txt"

#f = open(file_to_open)

#print(f.read())

#Read Images
#Read and prepare file lists
#Read files
imlist = {}
count = 0
#Windows Users:
#path = Path(r"C:\Users\tiago\Documents\GitHub\currenctRecon_VCOMP\code\dataset\train")
#path = r"C:\Users\tiago\Documents\GitHub\currenctRecon_VCOMP\code\dataset\train"
#imlist = {}
#count = 0
#for each in glob(path + "\\*"):
    #word = each.split("\\")[-1]
    #print(" #### Reading image category ", word, " ##### ")
    #imlist[word] = []
    #for imagefile in glob(path+"\\"+word+"\\*"):
        #print("Reading file ", imagefile)
        #im = cv2.imread(imagefile)
        #imlist[word].append(im)
        #count +=1 

#Unix Users
imlist = {}
count = 0
path = "dataset/train/"
for each in glob(path + "*"):
    word = each.split("/")[-1]
    print(" #### Reading image category ", word, " ##### ")
    imlist[word] = []
    for imagefile in glob(path+word+"/*"):
        print("Reading file ", imagefile)
        im = cv2.imread(imagefile)
        imlist[word].append(im)
        count +=1 
#self.images, self.trainImageCount = self.file_helper.getFiles(self.train_path)
#for dirName, subdirList, fileList in os.walk(path):
    #print('Found directory: %s' % dirName.split("/"))
    #for fname in fileList:
        #print('\t%s' % fname)
#dirName


 #### Reading image category  100Euro  ##### 
Reading file  dataset/train/100Euro/100eu_back.jpg
Reading file  dataset/train/100Euro/100eu_back_l.jpg
Reading file  dataset/train/100Euro/100eu_back_m.jpg
Reading file  dataset/train/100Euro/100eu_back_vl.jpg
Reading file  dataset/train/100Euro/100eu_front.jpg
Reading file  dataset/train/100Euro/100eu_front_l.jpg
Reading file  dataset/train/100Euro/100eu_front_m.jpg
Reading file  dataset/train/100Euro/100eu_front_vl.jpg
 #### Reading image category  10Euro  ##### 
Reading file  dataset/train/10Euro/10eu_back.jpg
Reading file  dataset/train/10Euro/10eu_back_l.jpg
Reading file  dataset/train/10Euro/10eu_back_m.jpg
Reading file  dataset/train/10Euro/10eu_back_vl.jpg
Reading file  dataset/train/10Euro/10eu_front.jpg
Reading file  dataset/train/10Euro/10eu_front_l.jpg
Reading file  dataset/train/10Euro/10eu_front_m.jpg
Reading file  dataset/train/10Euro/10eu_front_vl.jpg
 #### Reading image category  200Euro  ##### 
Reading file  dataset/train

In [13]:
#Pre-process  and feature computing functions
def preprocess(image):
    blur = cv2.bilateralFilter(image,8,16,12)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cll = clahe.apply(blur)
    return cll


def gray(image):
    g_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return g_img

def features(image):
    feat = cv2.xfeatures2d.SIFT_create() 
    keypoints, descriptors = feat.detectAndCompute(image, None)
    return keypoints, descriptors

In [14]:
#Pre-process images and extract features
label_count = 0
name_dict = {}
train_labels = np.array([])
descriptor_list = []
for word in imlist.keys():
    name_dict[str(label_count)] = word
    print ("Computing Features for ", word)
    for im in imlist[word]:
        #cv2.imshow("im", im)
        # cv2.waitKey()
        train_labels = np.append(train_labels, label_count)
        gray_img = gray(im)
        preprocessed_img = preprocess(gray_img)
        kp, des = features(preprocessed_img)
        descriptor_list.append(des)
    label_count += 1

Computing Features for  100Euro
Computing Features for  10Euro
Computing Features for  200Euro
Computing Features for  20Euro
Computing Features for  500Euro
Computing Features for  50Euro
Computing Features for  5Euro


In [26]:
#Perform clustering
#Let's create first the feature array, with the descriptors
vStack = np.array(descriptor_list[0])
for remaining in descriptor_list[1:]:
    vStack = np.vstack((vStack, remaining))
    descriptor_vstack = vStack.copy()
    #return vStack

#Initializa KMeans Classifier
#Number of Clusters for KMeans Classifier
n_clusters = 100
clf = KMeans(n_clusters = n_clusters)
kmean_predictions = clf.fit_predict(descriptor_vstack)

#Create a Vocabulary of Visual Words
n_images = count
mega_histogram = np.array([np.zeros(n_clusters) for i in range(n_images)])
old_count = 0
for i in range(n_images):
    l = len(descriptor_list[i])
    for j in range(l):
        if kmean_predictions is None:
            idx = kmean_predictions[old_count+j]
        else:
            idx = kmean_predictions[old_count+j]
        mega_histogram[i][idx] += 1
    old_count += l
print("Vocabulary Histogram Generated")




#self.bov_helper.developVocabulary(n_images = self.trainImageCount, descriptor_list=self.descriptor_list)

# show vocabulary trained

# self.bov_helper.plotHist()

Vocabulary Histogram Generated


In [27]:
import pickle
pickle.dump( mega_histogram, open( "mega_histogram_100.p", "wb" ) )

In [34]:
#Rescale histogram and normalize features
#self.bov_helper.standardize()
scale = StandardScaler().fit(mega_histogram)
mega_histogram = scale.transform(mega_histogram)

#Train a classifier: in this case we are using a Support Vector Machine
#Initialize SVM Classifier
clf  = SVC()
print("Training SVM")
print(clf)
print("Train labels", train_labels)
clf.fit(mega_histogram, train_labels)
print("Training completed")
#self.bov_helper.train(self.train_labels)

Training SVM
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Train labels [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
Training completed


In [41]:
clf.predict(mega_histogram)
mega_histogram.shape

(64, 100)

In [38]:
import numpy as np
from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=100000, tol=1e-3)
clf.fit(mega_histogram, train_labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge',
       max_iter=100000, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [39]:
clf.predict(mega_histogram)

array([0., 6., 6., 6., 0., 6., 6., 6., 1., 6., 6., 6., 1., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 0., 6., 6., 6., 3., 6., 6., 6., 4., 6.,
       6., 6., 4., 6., 6., 6., 5., 6., 6., 6., 5., 6., 6., 6., 6., 0., 6.,
       6., 6., 6., 6., 6., 6., 5., 6., 6., 6., 6., 6., 6.])

In [42]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(mega_histogram, train_labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [43]:
clf.predict(mega_histogram)

array([0., 6., 6., 6., 0., 6., 6., 6., 1., 6., 6., 6., 1., 6., 6., 6., 6.,
       6., 6., 6., 2., 6., 6., 6., 3., 6., 6., 6., 3., 6., 6., 6., 4., 6.,
       6., 6., 4., 6., 6., 6., 5., 6., 6., 6., 5., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 1., 6., 6., 6., 6., 6., 6.])

(100,)